Web Scraping data assignment. Toronto neighbourhood clusters.

Url = https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M of website.

In [1]:
import pandas as pd
import numpy as np
#webscraping imports
from bs4 import BeautifulSoup
import datetime
from tinydb import TinyDB, Query
import urllib3
import xlsxwriter

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
def make_soup(url):
    http = urllib3.PoolManager()
    r = http.request("GET", url)
    return BeautifulSoup(r.data,'lxml')

In [87]:
col_names = ["Postal Code", "Borough", "Neighbourhood"]

neighbourhoods = pd.DataFrame(columns = col_names)
neighbourhoods

,Postal Code,Borough,Neighbourhood


In [88]:
soup = make_soup(url)
# htmlfile

/Users/DRMair/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [89]:
result = soup.find_all("table", class_ = "wikitable sortable")

In [90]:
result_1 = result
type(result_1[0])


# soup = BeautifulSoup(result_1[0], 'lxml') # Parse the HTML as a string

bs4.element.Tag

In [91]:
table_rows = result[0].find_all('tr')

In [92]:
list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
#     neighbourhoods.iat[row_marker,column_marker] = column.get_text()
    list.append(row)

list.remove([])

In [93]:
for i in range(0, len(list)):
    for j in range(0,3):
        list[i][j] = list[i][j].strip('\n')

neighbourhoods = pd.DataFrame(list,columns = col_names)


In [94]:
neighbourhoods.dropna(axis = 0,how = 'any',inplace = True)

In [95]:
neighbourhoods = neighbourhoods[neighbourhoods.Borough != 'Not assigned']

In [96]:
neighbourhoods

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [99]:
neighbourhoods = neighbourhoods.reset_index()
neighbourhoods.drop(['level_0','index'], axis = 1, inplace= True)

In [100]:
neighbourhoods

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


We have the scraped data frame with Toronto postal codes, now we will add latitudes and longitudes to the data frame using geocoder csv file.

In [36]:
lat_lng = pd.read_csv("/Users/DRMair/Downloads/Geospatial_Coordinates.csv")

In [101]:
neighbourhoods = neighbourhoods.merge(lat_lng, on = 'Postal Code' )

In [102]:
neighbourhoods

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
